# スコアシートの総当たり閾値検索に加えて，相関係数も算出する
## なぜ？
## ＝閾値をクリアした同一comのモデルの中で，最終的に使うモデルを一つに絞るため

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt

import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
from sklearn.model_selection import train_test_split
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
import time
import datetime
from dateutil.relativedelta import relativedelta
import os #ディレクトリ作成用
import xgboost as xgb
import sys
#自作のモジュールのインポート
sys.path.append("..")
import module.master as master
import module.modeling_scores as modeling


### 方針
### ＝４つの区間をすべて使う，閾値の検索は無しとして決めのパラメータだけで絞り，その後各変数とfinal_gainとの相関係数を算出して，最終的に標準化した偏差値と組み合わせる

In [12]:
# 芦屋だけでひとまず機能を完成させる
#きめのパラメータ
place_name='asiya'
plus_m_th=2
num_hit_th=5
versions=['V4_1','V4_2','V4_3']
modes=['asiya_model_score_','asiya_model_score_period_']#モードは直近三カ月と，昨年同月の二種類
mode_types=['recent','period']#モードは直近三カ月と，昨年同月の二種類,modesと一緒にzipで回す

#for version in versions:
version='V4_2'
version_df=pd.DataFrame()#区間ごとのスコアシートの結合先（バージョンごとのまとめ）
#for mode,mode_type in zip(modes,mode_types):
mode=modes[0]
mode_type=mode_types[0]
for i in range(4):
    now_ym=datetime.datetime(year=2020, month=1+(i*3),day=1).strftime('%Y%m%d')
    dir_path='../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{mode}st{now_ym}_finalM3_{version}.csv'.format(place_name=place_name,mode=mode,now_ym=now_ym,version=version)
    model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')
    model_score_df=model_score_df.drop(["Unnamed: 0"],axis=1)
     #決めのパラメータでいったん切り落とし
    model_score_df=model_score_df[model_score_df['plus_month_num_test']>=plus_m_th].copy()#収益がプラスだった月の数で切り捨てる
    
    model_score_df=model_score_df[model_score_df['num_hit_test']>=num_hit_th].copy()#的中した予測の数で切り捨てる，大体一カ月に一件くらいをイメージ
    version_df= pd.concat([version_df, model_score_df], axis=0)#きめのパラメータで切り落としたので結合
    display(version_df)
#相関係数の算出（範囲は-1～1）
version_corr_df=version_df.corr()

,target_com,depth,target_per,threshold,total_get_test,total_use_test,num_com_test,num_pred_test,gain_test,gain_std_test,...,total_use_final,num_com_final,num_pred_final,gain_final,gain_std_final,num_hit_final,buy_hit_per_final,buy_hit_per_std_final,plus_month_num_final,diff_mea_med_final
0,1.0,5.0,81.0,0.50,50180.0,50700.0,69.0,507.0,98.974359,23.749464,...,31200.0,35.0,312.0,89.166667,37.234714,32.0,10.256410,0.806606,1.0,39.391026
1,1.0,5.0,81.0,0.54,47480.0,47800.0,69.0,478.0,99.330544,18.819784,...,28500.0,35.0,285.0,93.614035,42.330960,31.0,10.877193,1.071948,1.0,33.736264
2,1.0,5.0,81.0,0.58,42380.0,43200.0,69.0,432.0,98.101852,39.850919,...,24400.0,35.0,244.0,103.360656,35.993902,30.0,12.295082,1.084324,2.0,29.206349
3,1.0,5.0,82.0,0.50,50180.0,51100.0,69.0,511.0,98.199609,23.514959,...,31300.0,35.0,313.0,88.881789,37.349840,32.0,10.223642,0.804029,1.0,39.391026
4,1.0,5.0,82.0,0.54,47480.0,48200.0,69.0,482.0,98.506224,18.650926,...,28300.0,35.0,283.0,94.275618,42.962409,31.0,10.954064,1.079523,1.0,33.736264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11340,23.0,7.0,81.0,0.50,38110.0,40500.0,19.0,405.0,94.098765,94.271677,...,23700.0,9.0,237.0,149.367089,87.979899,8.0,3.375527,0.644525,2.0,1353.888889
11343,23.0,7.0,82.0,0.50,31520.0,41100.0,19.0,411.0,76.690998,66.085641,...,22900.0,9.0,229.0,144.410480,84.105870,7.0,3.056769,0.667042,2.0,28.333333
11349,23.0,7.0,84.0,0.50,31520.0,40300.0,19.0,403.0,78.213400,64.834589,...,23100.0,9.0,231.0,153.246753,93.104464,8.0,3.463203,0.661266,2.0,1353.888889
11352,23.0,7.0,85.0,0.50,31520.0,36900.0,19.0,369.0,85.420054,70.147520,...,22900.0,9.0,229.0,144.410480,84.105870,7.0,3.056769,0.667042,2.0,28.333333


,target_com,depth,target_per,threshold,total_get_test,total_use_test,num_com_test,num_pred_test,gain_test,gain_std_test,...,total_use_final,num_com_final,num_pred_final,gain_final,gain_std_final,num_hit_final,buy_hit_per_final,buy_hit_per_std_final,plus_month_num_final,diff_mea_med_final
0,1.0,5.0,81.0,0.50,50180.0,50700.0,69.0,507.0,98.974359,23.749464,...,31200.0,35.0,312.0,89.166667,37.234714,32.0,10.256410,0.806606,1.0,39.391026
1,1.0,5.0,81.0,0.54,47480.0,47800.0,69.0,478.0,99.330544,18.819784,...,28500.0,35.0,285.0,93.614035,42.330960,31.0,10.877193,1.071948,1.0,33.736264
2,1.0,5.0,81.0,0.58,42380.0,43200.0,69.0,432.0,98.101852,39.850919,...,24400.0,35.0,244.0,103.360656,35.993902,30.0,12.295082,1.084324,2.0,29.206349
3,1.0,5.0,82.0,0.50,50180.0,51100.0,69.0,511.0,98.199609,23.514959,...,31300.0,35.0,313.0,88.881789,37.349840,32.0,10.223642,0.804029,1.0,39.391026
4,1.0,5.0,82.0,0.54,47480.0,48200.0,69.0,482.0,98.506224,18.650926,...,28300.0,35.0,283.0,94.275618,42.962409,31.0,10.954064,1.079523,1.0,33.736264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11439,23.0,7.0,114.0,0.50,33830.0,22000.0,20.0,220.0,153.772727,116.302219,...,15600.0,6.0,156.0,14.038462,24.315329,1.0,0.641026,0.370096,0.0,0.000000
11442,23.0,7.0,115.0,0.50,19400.0,21600.0,20.0,216.0,89.814815,94.916323,...,15900.0,6.0,159.0,13.773585,23.856549,1.0,0.628931,0.363113,0.0,0.000000
11445,23.0,7.0,116.0,0.50,18000.0,19900.0,20.0,199.0,90.452261,114.040482,...,15500.0,6.0,155.0,14.129032,23.414761,1.0,0.645161,0.372484,0.0,0.000000
11454,23.0,7.0,119.0,0.50,18020.0,19200.0,20.0,192.0,93.854167,91.258517,...,15100.0,6.0,151.0,14.503311,23.856549,1.0,0.662252,0.382351,0.0,0.000000


,target_com,depth,target_per,threshold,total_get_test,total_use_test,num_com_test,num_pred_test,gain_test,gain_std_test,...,total_use_final,num_com_final,num_pred_final,gain_final,gain_std_final,num_hit_final,buy_hit_per_final,buy_hit_per_std_final,plus_month_num_final,diff_mea_med_final
0,1.0,5.0,81.0,0.50,50180.0,50700.0,69.0,507.0,98.974359,23.749464,...,31200.0,35.0,312.0,89.166667,37.234714,32.0,10.256410,0.806606,1.0,39.391026
1,1.0,5.0,81.0,0.54,47480.0,47800.0,69.0,478.0,99.330544,18.819784,...,28500.0,35.0,285.0,93.614035,42.330960,31.0,10.877193,1.071948,1.0,33.736264
2,1.0,5.0,81.0,0.58,42380.0,43200.0,69.0,432.0,98.101852,39.850919,...,24400.0,35.0,244.0,103.360656,35.993902,30.0,12.295082,1.084324,2.0,29.206349
3,1.0,5.0,82.0,0.50,50180.0,51100.0,69.0,511.0,98.199609,23.514959,...,31300.0,35.0,313.0,88.881789,37.349840,32.0,10.223642,0.804029,1.0,39.391026
4,1.0,5.0,82.0,0.54,47480.0,48200.0,69.0,482.0,98.506224,18.650926,...,28300.0,35.0,283.0,94.275618,42.962409,31.0,10.954064,1.079523,1.0,33.736264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12429,25.0,7.0,108.0,0.50,28420.0,25600.0,17.0,256.0,111.015625,151.391131,...,15000.0,2.0,150.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
12438,25.0,7.0,111.0,0.50,28420.0,24600.0,17.0,246.0,115.528455,149.819340,...,14600.0,2.0,146.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
12444,25.0,7.0,113.0,0.50,28420.0,23600.0,17.0,236.0,120.423729,157.258882,...,13400.0,2.0,134.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
12450,25.0,7.0,115.0,0.50,28420.0,23000.0,17.0,230.0,123.565217,163.545772,...,13400.0,2.0,134.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000


,target_com,depth,target_per,threshold,total_get_test,total_use_test,num_com_test,num_pred_test,gain_test,gain_std_test,...,total_use_final,num_com_final,num_pred_final,gain_final,gain_std_final,num_hit_final,buy_hit_per_final,buy_hit_per_std_final,plus_month_num_final,diff_mea_med_final
0,1.0,5.0,81.0,0.50,50180.0,50700.0,69.0,507.0,98.974359,23.749464,...,31200.0,35.0,312.0,89.166667,37.234714,32.0,10.256410,0.806606,1.0,39.391026
1,1.0,5.0,81.0,0.54,47480.0,47800.0,69.0,478.0,99.330544,18.819784,...,28500.0,35.0,285.0,93.614035,42.330960,31.0,10.877193,1.071948,1.0,33.736264
2,1.0,5.0,81.0,0.58,42380.0,43200.0,69.0,432.0,98.101852,39.850919,...,24400.0,35.0,244.0,103.360656,35.993902,30.0,12.295082,1.084324,2.0,29.206349
3,1.0,5.0,82.0,0.50,50180.0,51100.0,69.0,511.0,98.199609,23.514959,...,31300.0,35.0,313.0,88.881789,37.349840,32.0,10.223642,0.804029,1.0,39.391026
4,1.0,5.0,82.0,0.54,47480.0,48200.0,69.0,482.0,98.506224,18.650926,...,28300.0,35.0,283.0,94.275618,42.962409,31.0,10.954064,1.079523,1.0,33.736264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12868,26.0,7.0,86.0,0.54,31110.0,34200.0,8.0,342.0,90.964912,75.481018,...,16800.0,6.0,168.0,144.464286,269.467135,1.0,0.595238,0.343661,1.0,0.000000
12870,26.0,7.0,87.0,0.50,31110.0,41100.0,8.0,411.0,75.693431,66.859040,...,20800.0,6.0,208.0,136.009615,202.343450,2.0,0.961538,0.277572,1.0,0.000000
12873,26.0,7.0,88.0,0.50,31110.0,39100.0,8.0,391.0,79.565217,71.396181,...,20800.0,6.0,208.0,136.009615,201.887440,2.0,0.961538,0.277572,1.0,0.000000
12882,26.0,7.0,91.0,0.50,31110.0,37900.0,8.0,379.0,82.084433,71.191872,...,20200.0,6.0,202.0,120.148515,226.004694,1.0,0.495050,0.285817,1.0,0.000000


In [7]:
version_corr_df['gain_final']

target_com               0.082694
depth                    0.016542
target_per               0.118569
threshold                0.008366
total_get_test          -0.123978
total_use_test          -0.195773
num_com_test            -0.121898
num_pred_test           -0.195773
gain_test                0.131905
gain_std_test            0.134585
num_hit_test            -0.092406
buy_hit_per_test         0.092305
buy_hit_per_std_test     0.259471
plus_month_num_test      0.120055
diff_mea_med_test       -0.231564
total_get_final          0.665401
total_use_final         -0.179305
num_com_final            0.006614
num_pred_final          -0.179305
gain_final               1.000000
gain_std_final           0.684599
num_hit_final            0.119247
buy_hit_per_final        0.254617
buy_hit_per_std_final    0.093479
plus_month_num_final     0.681998
diff_mea_med_final       0.113812
Name: gain_final, dtype: float64

In [8]:
model_score_df.to_csv("ex.csv")

In [9]:
version_df

,target_com,depth,target_per,threshold,total_get_test,total_use_test,num_com_test,num_pred_test,gain_test,gain_std_test,...,total_use_final,num_com_final,num_pred_final,gain_final,gain_std_final,num_hit_final,buy_hit_per_final,buy_hit_per_std_final,plus_month_num_final,diff_mea_med_final
0,1.0,5.0,81.0,0.50,50180.0,50700.0,69.0,507.0,98.974359,23.749464,...,31200.0,35.0,312.0,89.166667,37.234714,32.0,10.256410,0.806606,1.0,39.391026
1,1.0,5.0,81.0,0.54,47480.0,47800.0,69.0,478.0,99.330544,18.819784,...,28500.0,35.0,285.0,93.614035,42.330960,31.0,10.877193,1.071948,1.0,33.736264
2,1.0,5.0,81.0,0.58,42380.0,43200.0,69.0,432.0,98.101852,39.850919,...,24400.0,35.0,244.0,103.360656,35.993902,30.0,12.295082,1.084324,2.0,29.206349
3,1.0,5.0,82.0,0.50,50180.0,51100.0,69.0,511.0,98.199609,23.514959,...,31300.0,35.0,313.0,88.881789,37.349840,32.0,10.223642,0.804029,1.0,39.391026
4,1.0,5.0,82.0,0.54,47480.0,48200.0,69.0,482.0,98.506224,18.650926,...,28300.0,35.0,283.0,94.275618,42.962409,31.0,10.954064,1.079523,1.0,33.736264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12868,26.0,7.0,86.0,0.54,31110.0,34200.0,8.0,342.0,90.964912,75.481018,...,16800.0,6.0,168.0,144.464286,269.467135,1.0,0.595238,0.343661,1.0,0.000000
12870,26.0,7.0,87.0,0.50,31110.0,41100.0,8.0,411.0,75.693431,66.859040,...,20800.0,6.0,208.0,136.009615,202.343450,2.0,0.961538,0.277572,1.0,0.000000
12873,26.0,7.0,88.0,0.50,31110.0,39100.0,8.0,391.0,79.565217,71.396181,...,20800.0,6.0,208.0,136.009615,201.887440,2.0,0.961538,0.277572,1.0,0.000000
12882,26.0,7.0,91.0,0.50,31110.0,37900.0,8.0,379.0,82.084433,71.191872,...,20200.0,6.0,202.0,120.148515,226.004694,1.0,0.495050,0.285817,1.0,0.000000


In [10]:
model_score_df

,target_com,depth,target_per,threshold,total_get_test,total_use_test,num_com_test,num_pred_test,gain_test,gain_std_test,...,total_use_final,num_com_final,num_pred_final,gain_final,gain_std_final,num_hit_final,buy_hit_per_final,buy_hit_per_std_final,plus_month_num_final,diff_mea_med_final
1,1.0,5.0,81.0,0.54,42950.0,53400.0,75.0,534.0,80.430712,29.342666,...,29200.0,54.0,292.0,118.527397,64.244615,34.0,11.643836,1.544264,2.0,234.661797
4,1.0,5.0,82.0,0.54,42950.0,53700.0,75.0,537.0,79.981378,29.549551,...,29000.0,54.0,290.0,119.344828,65.074719,34.0,11.724138,1.554914,2.0,234.661797
7,1.0,5.0,83.0,0.54,42950.0,53000.0,75.0,530.0,81.037736,29.023256,...,28900.0,54.0,289.0,119.757785,65.755125,34.0,11.764706,1.560294,2.0,234.661797
10,1.0,5.0,84.0,0.54,42950.0,53100.0,75.0,531.0,80.885122,29.079693,...,28700.0,54.0,287.0,120.592334,65.630090,34.0,11.846690,1.571167,2.0,234.661797
11,1.0,5.0,84.0,0.58,36400.0,47700.0,75.0,477.0,76.310273,35.199246,...,25200.0,54.0,252.0,108.293651,44.271533,30.0,11.904762,1.587302,2.0,172.253968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12868,26.0,7.0,86.0,0.54,31110.0,34200.0,8.0,342.0,90.964912,75.481018,...,16800.0,6.0,168.0,144.464286,269.467135,1.0,0.595238,0.343661,1.0,0.000000
12870,26.0,7.0,87.0,0.50,31110.0,41100.0,8.0,411.0,75.693431,66.859040,...,20800.0,6.0,208.0,136.009615,202.343450,2.0,0.961538,0.277572,1.0,0.000000
12873,26.0,7.0,88.0,0.50,31110.0,39100.0,8.0,391.0,79.565217,71.396181,...,20800.0,6.0,208.0,136.009615,201.887440,2.0,0.961538,0.277572,1.0,0.000000
12882,26.0,7.0,91.0,0.50,31110.0,37900.0,8.0,379.0,82.084433,71.191872,...,20200.0,6.0,202.0,120.148515,226.004694,1.0,0.495050,0.285817,1.0,0.000000
